In [1]:
import pypsa
import pandas as pd

In [2]:
network = pypsa.Network()

# Create an empty network.
network.set_snapshots(range(4))

# Add a bus to the network.
network.add("Bus","bus")

# Add a coal power plant to the bus.
network.add("Generator","coal",bus="bus",
       committable=True,
       p_min_pu=0.3,
       marginal_cost=20,
       p_nom=10000)

# Add a gas power plant to the bus.
network.add("Generator","gas",bus="bus",
       committable=True,
       marginal_cost=25,
       p_min_pu=0.1,
       p_nom=1000)

# Add a 2nd gas power plant to the bus.
network.add("Generator","gas2",bus="bus",
       committable=True,
       marginal_cost=30,
       p_min_pu=0.1,
       p_nom=2000)

# Add load to the bus.
network.add("Load","load",bus="bus",p_set=[4000,6000,5000,800])

In [3]:
# Solve the network.
network.lopf(network.snapshots,solver_name='gurobi')

INFO:pypsa.pf:Slack bus for sub-network 0 is bus
INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `angles` formulation
INFO:pypsa.opf:Solving model using gurobi
INFO:pypsa.opf:Optimization successful


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x29
  Lower bound: 320000.0
  Upper bound: 320000.0
  Number of objectives: 1
  Number of constraints: 33
  Number of variables: 29
  Number of binary variables: 12
  Number of integer variables: 12
  Number of continuous variables: 17
  Number of nonzeros: 65
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination message: Model was solved to optimalit

('ok', 'optimal')

In [4]:
# Print the dispatch of each generator.
network.generators_t.p

,coal,gas,gas2
0,4000.0,0.0,0.0
1,6000.0,0.0,0.0
2,5000.0,0.0,0.0
3,0.0,800.0,0.0


## Below here is MGA stuff


In [5]:
import pyomo.environ as pyomo_env

# Saving the value of the old objective function.
old_objective_value = network.model.objective()
print(old_objective_value)


320000.0


In [6]:
def extra_functionality(network, snapshots,  MGA_slack = 0.05):

    # Identify the nonzero decision variables that should enter the MGA objective function.
    generator_outputs = network.generators_t.p
    nonzero_gen_p = list()
    for gen_p in network.model.generator_p :
        if generator_outputs[gen_p[0]].loc[gen_p[1]] > 0 :
            nonzero_gen_p.append(gen_p)
    
    # Build new MGA objective function.
    MGA_objective = 0
    for gen_p in nonzero_gen_p:
        MGA_objective += network.model.generator_p[gen_p]

    # Add the new MGA objective function to the model.
    network.model.mga_objective = pyomo_env.Objective(expr=MGA_objective)

    # Deactivate the old objective function and activate the MGA objective function.
    network.model.objective.deactivate()
    network.model.mga_objective.activate()

    # Add the MGA slack constraint.
    #print('old objective value ',old_objective_value)
    network.model.mga_constraint = pyomo_env.Constraint(expr=network.model.objective.expr <= 
                                          (1 + MGA_slack) * old_objective_value)

# Itterating

In [11]:
# Defining a list for solutions
network.solutions = []
MGA_slack = 0.1
# Loop until a non original solution is found
while True:
    network.lopf(network.snapshots,solver_name='gurobi',extra_functionality=lambda network, snapshots: extra_functionality(network, snapshots, MGA_slack)) 
    # Tjek if the solution is seen before, otherwise add solution to list of solutions
    for i in range(len(network.solutions)):
        if network.generators_t.p.equals(network.solutions[i]):
            break
    else : 
        network.solutions.append(network.generators_t.p)
        continue
    break
            
        
 

INFO:pypsa.pf:Slack bus for sub-network 0 is bus
INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `angles` formulation
INFO:pypsa.opf:Solving model using gurobi
INFO:pypsa.opf:Optimization successful
INFO:pypsa.pf:Slack bus for sub-network 0 is bus
INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `angles` formulation
INFO:pypsa.opf:Solving model using gurobi


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x29
  Lower bound: 10800.0
  Upper bound: 10800.0
  Number of objectives: 1
  Number of constraints: 34
  Number of variables: 29
  Number of binary variables: 12
  Number of integer variables: 12
  Number of continuous variables: 17
  Number of nonzeros: 77
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination message: Model was solved to optimality 

INFO:pypsa.opf:Optimization successful
INFO:pypsa.pf:Slack bus for sub-network 0 is bus
INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `angles` formulation
INFO:pypsa.opf:Solving model using gurobi


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x29
  Lower bound: 12000.0
  Upper bound: 12000.0
  Number of objectives: 1
  Number of constraints: 34
  Number of variables: 29
  Number of binary variables: 12
  Number of integer variables: 12
  Number of continuous variables: 17
  Number of nonzeros: 77
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination message: Model was solved to optimality 

INFO:pypsa.opf:Optimization successful


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x29
  Lower bound: 10800.0
  Upper bound: 10800.0
  Number of objectives: 1
  Number of constraints: 34
  Number of variables: 29
  Number of binary variables: 12
  Number of integer variables: 12
  Number of continuous variables: 17
  Number of nonzeros: 77
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination message: Model was solved to optimality 

In [12]:
# Print found solutions
for i in range(len(network.solutions)):
    print('solution ',i+1)
    print(network.solutions[i])


solution  1
     coal     gas    gas2
0  3800.0     0.0   200.0
1  3000.0  1000.0  2000.0
2  4000.0  1000.0     0.0
3     0.0   800.0     0.0
solution  2
     coal     gas    gas2
0  3000.0  1000.0     0.0
1  6000.0     0.0     0.0
2  3000.0     0.0  2000.0
3     0.0     0.0   800.0
